In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Input

class Task:
    def __init__(self, id, power, start_time, duration):
        self.id = id
        self.power = power
        self.start_time = start_time
        self.duration = duration

    def adjust_priority(self, weight):
        self.priority = max(0.1, min(1, weight))  # Ensure priority stays within [0.1, 1]

def simulate_consumption(tasks, time, dt):
    consumption = np.zeros_like(time)
    for task in tasks:
        idx_range = slice(int(task.start_time / dt), int((task.start_time + task.duration) / dt))
        consumption[idx_range] += task.power * getattr(task, 'priority', 1)
    return consumption

def available_power(t, phase, k, efficiency, max_panel_power, t_final):
    return max(max_panel_power * np.cos(2 * np.pi * t / t_final + phase) * efficiency * k, 0)

def manage_soc(time, power, consumption, initial_soc, max_battery_power, dt, nominal_consumption):
    soc = np.zeros_like(time)
    soc[0] = initial_soc
    for i in range(1, len(time)):
        energy_generated = power[i-1] * dt / 60
        soc[i] = max(0, min(max_battery_power, soc[i-1] + energy_generated - (nominal_consumption * dt / 60) - consumption[i-1]))
    return soc

def build_model(input_shape=(4,), output_activation='sigmoid'):
    model = Sequential([
        Input(shape=input_shape),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation=output_activation)
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def generate_synthetic_data(num_samples=1000, soc_initial=95, hard_deck=20):
    np.random.seed(42)
    df = pd.DataFrame({
        'StartTime': np.random.uniform(0, 100, num_samples),
        'Duration': np.random.uniform(1, 10, num_samples),
        'Power': np.random.uniform(1, 3, num_samples),
        'SoC': np.random.uniform(soc_initial-10, soc_initial+10, num_samples)
    })
    df['Execute'] = np.where(df['SoC'] > hard_deck, 1, 0)
    return df

def train_model(model, training_data):
    features = training_data[['StartTime', 'Duration', 'Power', 'SoC']].values
    targets = training_data['Execute'].values
    model.fit(features, targets, epochs=50, batch_size=32, verbose=0)

def execute_selected_task(tasks_df, current_soc, model, hard_deck):
    if tasks_df.empty:
        return None, current_soc
    features = tasks_df[['StartTime', 'Duration', 'Power']].assign(SoC=current_soc).values
    decisions = model.predict(features).flatten()
    tasks_df['Execute'] = decisions
    selected_task = tasks_df.sort_values(by='Execute', ascending=False).iloc[0]
    new_soc = max(current_soc - selected_task['Power'] * selected_task['Duration'], hard_deck)
    return selected_task, new_soc

# Parameters and Initialization
phase = np.random.rand() * 2 * np.pi
max_battery_power, max_panel_power = 43.68, 24.24
k, efficiency = 191.1, 21.528 / 100
time_initial, time_final, dt = 0.0, 97.0, 1.0
soc_initial, hard_deck, soft_deck = 95, 20, 30
nominal_consumption = 2

# Tasks DataFrame
tasks_df = pd.DataFrame({
    'TaskID': [1, 2, 3, 4],
    'Power': [1.5, 2.0, 1.0, 2.5],
    'StartTime': [10, 30, 50, 70],
    'Duration': [5, 1, 5, 1]
})

# Create and train the model
model = build_model()
training_data = generate_synthetic_data()
train_model(model, training_data)

# Simulation loop
times = np.arange(time_initial, time_final + dt, dt)
powers = [available_power(t, phase, k, efficiency, max_panel_power, time_final) for t in times]
current_soc = soc_initial

while not tasks_df.empty:
    selected_task, current_soc = execute_selected_task(tasks_df, current_soc, model, hard_deck)
    if selected_task is not None:
        print(f"Executing task {selected_task['TaskID']} with current SoC: {current_soc}%")
        tasks_df = tasks_df[tasks_df['TaskID'] != selected_task['TaskID']]
    else:
        print("No suitable tasks for execution.")
        break

# Visualizations and additional analyses can be added as required.


ModuleNotFoundError: No module named 'keras'